In [20]:
import requests as req
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import duckdb

### Creating NCAA stats dataframe

In [11]:
classes=['Fr','So','Jr','Sr']
years=np.arange(2003,2025,1)
columns=['player','cls','year','gp','mpg','ppg','fgm','fga','fg%','3pm','3pa','3p%','ftm','fta','ft%','orb',
        'drb','rpg','apg','spg','bpg','tov','pf']

In [12]:
table=[]
for year in tqdm(years):
    for cls in classes:
        for page in range(1,4):
            try:
                url=f'https://basketball.realgm.com/ncaa/stats/{year}/Averages/Qualified/{cls}/Season/All/points/desc/{page}'
                r=req.get(url)
                soup=BeautifulSoup(r.text)
                for i in range(1,31):
                    row=[str(soup.find_all('tr')[i]).split('</td>')[1].split('>')[-2][:-3],cls,year]
                    for j in range(3,23):
                        row.append(str(soup.find_all('tr')[i]).split('</td>')[j][4:])
                    table.append(row)
            except:
                print('error: '+url)
                pass

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [15:48<00:00, 43.13s/it]


In [13]:
df=pd.DataFrame(table,columns=columns)

In [7]:
df

,player,cls,year,gp,mpg,ppg,fgm,fga,fg%,3pm,...,fta,ft%,orb,drb,rpg,apg,spg,bpg,tov,pf
0,Kee-Kee Clark,Fr,2003,29,38.2,24.9,8.0,20.1,0.396,3.8,...,6.1,0.853,0.8,2.4,3.3,4.2,1.4,0.2,4.0,1.9
1,Carmelo Anthony,Fr,2003,35,36.4,22.2,7.9,17.5,0.453,1.6,...,6.8,0.706,2.9,6.9,9.7,2.2,1.5,0.9,2.2,2.2
2,Darshan Luckey,Fr,2003,28,37.5,21.6,7.2,18.8,0.383,1.6,...,7.2,0.777,0.8,4.4,5.1,1.6,1.7,0.3,4.1,2.3
3,Craig Smith,Fr,2003,31,31.9,19.9,7.8,12.9,0.603,0.1,...,6.3,0.677,3.2,4.7,7.9,1.3,0.9,0.9,2.4,3.4
4,Ike Diogu,Fr,2003,32,32.2,19.0,6.6,10.8,0.609,0.3,...,7.7,0.735,3.1,4.8,7.8,0.8,0.3,1.0,2.8,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,Gerald Drumgoole,Sr,2024,33,28.5,13.9,4.7,10.8,0.433,2.3,...,3.5,0.643,0.6,3.1,3.8,2.4,0.4,0.2,1.5,1.5
7916,Jestin Porter,Sr,2024,33,31.8,13.9,5.0,12.5,0.399,1.8,...,2.7,0.775,0.8,2.5,3.3,1.8,1.0,0.2,2.2,1.9
7917,Aaron Clarke,Sr,2024,31,33.1,13.9,4.9,12.0,0.411,2.0,...,2.6,0.775,0.4,3.2,3.5,3.0,0.7,0.1,1.3,1.5
7918,Brycen Goodine,Sr,2024,31,25.9,13.9,4.9,9.8,0.498,2.5,...,2.2,0.761,0.8,2.2,3.0,1.4,0.8,0.3,1.5,2.0


### Creating NBA drafts dataframe 

In [34]:
years=np.arange(2004,2024,1)

In [46]:
table=[]
for year in tqdm(years):
    try:
        url=f'https://basketball.realgm.com/nba/draft/past_drafts/{year}'
        r=req.get(url)
        soup=BeautifulSoup(r.text)
        for i in tqdm(range(1,31)):
            name1=str(soup.find_all('table')[12].find_all('tr')[i]).split('</a>')[0].split('>')[-1]
            name2=str(soup.find_all('table')[13].find_all('tr')[i]).split('</a>')[0].split('>')[-1]
            table.append([name1,year,i,'first round'])
            table.append([name2,year,i,'second round'])
    except:
        print('error: '+url,i,name1,name2)
        pass

  5%|████▏                                                                              | 1/20 [00:01<00:22,  1.21s/it]

error: https://basketball.realgm.com/nba/draft/past_drafts/2004 30 David Harrison Blake Stepp



 85%|█████████████████████████████████████████████████████████████████████▋            | 17/20 [00:18<00:03,  1.11s/it]


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:20<00:01,  1.10s/it]

error: https://basketball.realgm.com/nba/draft/past_drafts/2022 29 TyTy Washington, Jr. Hugo Besson



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:21<00:00,  1.06s/it]

error: https://basketball.realgm.com/nba/draft/past_drafts/2023 29 Julian Strawther Chris Livingston


In [47]:
df=pd.DataFrame(table,columns=['player','year','draft_pos','drafted'])

In [7]:
df

,player,year,draft_pos,drafted
0,Dwight Howard,2004,1,first round
1,Anderson Varejao,2004,1,second round
2,Emeka Okafor,2004,2,first round
3,Jackson Vroman,2004,2,second round
4,Ben Gordon,2004,3,first round
...,...,...,...,...
1185,Tarik Biberovic,2023,26,second round
1186,"Nick Smith, Jr.",2023,27,first round
1187,Trayce Jackson-Davis,2023,27,second round
1188,Brice Sensabaugh,2023,28,first round


In [21]:
df=duckdb.query("SELECT * FROM df ORDER BY year,drafted,draft_pos").df()
df

,player,year,draft_pos,drafted
0,Dwight Howard,2004,1,first round
1,Emeka Okafor,2004,2,first round
2,Ben Gordon,2004,3,first round
3,Shaun Livingston,2004,4,first round
4,Devin Harris,2004,5,first round
...,...,...,...,...
1185,Jalen Slawson,2023,24,second round
1186,Isaiah Wong,2023,25,second round
1187,Tarik Biberovic,2023,26,second round
1188,Trayce Jackson-Davis,2023,27,second round


In [18]:
df.to_excel('nba-past-drafts.xlsx',index=False)

### Joining both tables

In [22]:
stats=pd.read_excel('ncaa-stats-big.xlsx')
drafts=pd.read_excel('nba-past-drafts.xlsx')

In [26]:
conn = duckdb.connect(':memory:')

conn.register("stats", stats)
conn.register("drafts", drafts)

In [47]:
query = """
    SELECT stats.*,drafts.draft_pos,drafts.drafted FROM stats
    LEFT JOIN drafts ON stats.player = drafts.player
"""

In [48]:
df = conn.execute(query).fetchdf()

 TIRAR ENTRADAS NULAS DO DRAFT_POS E DRAFTED (BOTA

In [60]:
df.to_excel('table.xlsx',index=False)